# PV Secondary Target Selection

## sga_semimajor_ends
### Method to create the secondary targeting file for the end points on the semi-major axis of large galaxies (from the SGA) in the BGS sample

##### Author: Kelly Douglass (University of Rochester)

See `/project/projectdirs/desi/target/secondary/README` for output data model

### Target classes
1. **End points on the major axis**
2. Points along the major axis

In [1]:
from astropy.table import Table, Column

import numpy as np

import matplotlib.pyplot as plt
%matplotlib notebook

## Parameters

In [2]:
OVERRIDE = True
REF_EPOCH = 2015.5

output_directory = '/project/projectdirs/desi/target/secondary/indata/'

## Target catalogs

[Siena Galaxy Atlas](https://portal.nersc.gov/project/cosmo/temp/ioannis/SGA-html-dr9alpha/SGA-2arcmin.html)

In [ ]:
# Target catalog file names

input_directory = '/Users/kellydouglass/Documents/Research/data/SGA/'

input_filename = input_directory + 'SGA_BGS_inspecfootprint.fits'

large_galaxies = Table.read(input_filename, format='fits')

In [18]:
large_galaxies[['TYPE', 'SGA_TYPE', 'SGA_MORPHTYPE']][large_galaxies['inSGA']]

TYPE,SGA_TYPE,SGA_MORPHTYPE
bytes4,bytes4,bytes21
DEV,SER,Sb
DEV,SER,
DEV,DEV,
DEV,SER,
EXP,SER,
DEV,SER,E
DEV,SER,
DEV,SER,
COMP,SER,


In [21]:
np.unique(large_galaxies['SGA_TYPE'][large_galaxies['inSGA']])

DEV
EXP
REX
SER


## Set first priority: points at either end of the major axis

Note: Galaxy centers should already be BGS targets

### Inclination angle minimum

We require a minimum inclination angle of $25^\circ$.

In [4]:
i_min = 25 # degrees

i_boolean = large_galaxies['BA_ratio'] <= np.cos(i_min*np.pi/180.)

lg_to_observe = large_galaxies[i_boolean]

### Disk galaxies

We require an exponential profile fit from Tractor.

In [8]:
disk_to_observe = lg_to_observe[lg_to_observe['TYPE'] == 'EXP']

### Within the SGA

We require all galaxies to have a minimum diameter at the 25 magnitude of 20".

In [9]:
SGA_disks = disk_to_observe[disk_to_observe['inSGA']]

## Calculate (ra, dec) of major axis end points

We set two fiber target locations at either end of the major axis at a distance of $xR_e$, where $x$ will be determined based on the results of the measurements taken during SV.

In [12]:
center_ra = SGA_disks['RA']   # degrees
center_dec = SGA_disks['DEC'] # degrees

phi = SGA_disks['pos_angle']*np.pi/180. # degrees -> radians
r50 = SGA_disks['uncor_radius']/3600.   # arcsecs -> degrees

x = 2. # To be determined during SV

# Maximum distance along the semi-major axis from the center coordinate for our endpoints
delta_a = x*r50

# Target positions
fiber1_ra = center_ra - delta_a*np.sin(phi)
fiber1_dec = center_dec + delta_a*np.cos(phi)
fiber2_ra = center_ra + delta_a*np.sin(phi)
fiber2_dec = center_dec - delta_a*np.cos(phi)

fiber_ra = np.concatenate((fiber1_ra, fiber2_ra))
fiber_dec = np.concatenate((fiber1_dec, fiber2_dec))

### Write target list to file

In [13]:
N_targets = len(fiber_ra)

In [14]:
disk_edges = Table([Column(fiber_ra, name='RA'), 
                    Column(fiber_dec, name='DEC'), 
                    Column(np.zeros(N_targets, dtype='>f4'), name='PMRA'), 
                    Column(np.zeros(N_targets, dtype='>f4'), name='PMDEC'), 
                    Column(REF_EPOCH*np.ones(N_targets, dtype='>f4'), name='REF_EPOCH'),
                    Column(OVERRIDE*np.ones(N_targets, dtype='bool'), name='OVERRIDE')])

In [ ]:
LG_centers.write(output_directory + 'sga_semimajor_ends.fits', format='fits')

## Target statistics

In [15]:
print('The number of disk galaxies is', len(SGA_disks))
print('The number of targets is', N_targets)

sky_chunk_boolean = np.logical_and.reduce([disk_edges['RA'] > 150, disk_edges['RA'] < 250, 
                                           disk_edges['DEC'] > 0, disk_edges['DEC'] < 50])

num_targets_in_sky_chunk = np.sum(sky_chunk_boolean)

sky_area = 100*50

print('The number of fiber placements in this portion of the sky is', 
      num_targets_in_sky_chunk)
print('The number of fiber placements per square degree is', 
      num_targets_in_sky_chunk/sky_area)

The number of disk galaxies is 55758
The number of targets is 111516
The number of fiber placements in this portion of the sky is 41207
The number of fiber placements per square degree is 8.2414
